In [1]:
import sys
sys.path.append("../../smsl")

In [2]:
from smsl.graphBigtraj import BigTrajGraphAgent
from smsl.config import ConfAgent
from smsl.graph import RB

In [3]:
import os
import pandas as pd
import numpy as np
from IPython.display import JSON

In [4]:
class HotSpotAgent(dict, ConfAgent):
    def __init__(self, MAgent):
        ConfAgent.__init__(self)
        self.end_ns = self.time_num
        self.framesperns = self.frame_num // self.time_num
        self.MAgent = MAgent
    def load_systems(self, system2offical_name):
        ori_path = os.getcwd()
        for system, system_offical_name in system2offical_name.items():
            os.chdir(f'../../{system}/d.fluctmatch')
            b_agent = BigTrajGraphAgent(self.MAgent)
            b_agent.system_offical_name = system_offical_name
            self[system] = b_agent
        os.chdir(ori_path)
    def get_hotspot(self, verbose=0, is_return=False):
        self.m_abbr = self.get_m_abbr() 
        dict_hotspot = {self.m_abbr:{}}
        for system, b_agent in self.items():
            dict_hotspot[b_agent.m_abbr][system] = {}
            dict_hotspot[b_agent.m_abbr][system] = get_hotspot_by_graph_agent(b_agent.t_agent, n_bp=self.n_bp, verbose=verbose)
        if is_return:
            return dict_hotspot
        else:
            self.dict_hotspot = dict_hotspot
    def get_m_abbr(self):
        return list(self.values())[0].m_abbr ## first system of sub group name
    def get_df_k_with_hotspots(self, k_value='k_avg_std', is_return=False): ## for s = pr0
        list_k_with_hotspots = []
        for s, dict_subgroup in self.dict_hotspot.items(): 
            for system, dict_system in dict_subgroup.items():
                for position, dict_position in dict_system.items():
                    pos_seq = position
                    k_avg = dict_position['hotspot']['avg']['avg']
                    k_std = dict_position['hotspot']['avg']['std']
                    list_k_with_hotspots.append({
                        'system'   : self[system].system_offical_name,
                        'sequence' : pos_seq,
                        'k_avg'    : k_avg,
                        'k_std'    : k_std,
                        'k_avg_std': f'{k_avg:.2f} ± {k_std:.2f}',
                    })
        df_k_with_hotspots_1 = pd.DataFrame(list_k_with_hotspots)
        df_k_with_hotspots = df_k_with_hotspots_1.set_index(['sequence'])
        df_k_with_hotspots = df_k_with_hotspots.pivot(columns='system', values=k_value)

        ## sort the row and column index
        sort_system  = list(dict.fromkeys(df_k_with_hotspots_1['system']))
        sort_seq     = list(dict.fromkeys(df_k_with_hotspots_1['sequence']))
        df_k_with_hotspots = df_k_with_hotspots.loc[sort_seq, sort_system]
        if is_return:
            return df_k_with_hotspots
        else:
            self.df_k_with_hotspots = df_k_with_hotspots 

In [5]:
def generate_cluster_hotspots(self): ## for s = rb
    cluster2hotspots = {
        'T': {i:['rank',i] for i in range(1, 5+1)}, ## rank1..rank5
        'C': {i:['rank',i] for i in range(1, 5+1)}, ## rank1..rank5
        'A': {i:['rank',i] for i in range(1, 5+1)}, ## rank1..rank5
        'G': {i:['rank',i] for i in range(1, 5+1)}, ## rank1..rank5
    }
    return cluster2hotspots

## SmallTimeAgent level
def get_cluster(self, resid_i, resid_j): ## for s = rb
    resname_i = self.strandid2sequence['STRAND1'][resid_i]
    resname_j = self.strandid2sequence['STRAND1'][resid_j]
    if resname_i != resname_j:
        raise RuntimeError('resname_i ≠ resname_j, resid_i:', resid_i, 'resid_j:', resid_j)
    cluster = resname_i
    return cluster

## SmallTimeAgent level
def generate_df_position2hotspots(self, resid_i, resid_j, hotspots): ## for s = rb
    dict_ks = {}
    df_m = self.mean.df_m
    mask1 = (df_m['Resid_i']==resid_i) & (df_m['Resid_j']==resid_j)
    df_resid_ij = df_m[mask1]
    for i, hotspot in hotspots.items():
        if isinstance(hotspot[1], int): ## rank n: ['rank', 1]
            df_sele = df_resid_ij.sort_values(by='k', ascending=False).iloc[[hotspot[1]-1]]
        elif isinstance(hotspot[1], str): ## hotspot: ['N1', 'O6']
            mask2 = (df_resid_ij['Atomname_i']==hotspot[0]) & (df_resid_ij['Atomname_j']==hotspot[1])
            df_sele = df_resid_ij[mask2]
        else:
            raise RuntimeError('Could not recognize cluster2hotspots:', cluster2hotspots)
        ## generate dict_ks
        pair_id, ser_sele = df_sele.index, df_sele.iloc[0]
        dict_ks[i] = {
            'query'     : ''.join(map(str, hotspot)),
            'atomname_i': ser_sele.Atomname_j,
            'atomname_j': ser_sele.Atomname_i,
            'k'         : {},
        }
        for time_label, st_agent in self.items():
            dict_ks[i]['k'][time_label] = st_agent.df_m.loc[pair_id].squeeze().k
        ks = pd.Series(dict_ks[i]['k'])
        dict_ks[i]['k']['avg'] = ks.mean()
        dict_ks[i]['k']['std'] = ks.std() ## Note: numpy.std(ddof=0), but pandas.std(ddof=1)
    ## k_avg in {aiaj}^s
    dict_ks['avg']  = {}
    for time_label, _ in self.items():
        ks = []
        for i, hotspot in hotspots.items():
            ks.append(dict_ks[i]['k'][time_label])
        dict_ks['avg'][time_label] = np.mean(ks)
    ks = pd.Series(dict_ks['avg'])
    dict_ks['avg']['avg'] = ks.mean()
    dict_ks['avg']['std'] = ks.std() ## Note: numpy.std(ddof=0), but pandas.std(ddof=1)
    return dict_ks

## GraphAgent level, for s = rb
def get_hotspot_by_graph_agent(self, n_bp, verbose=0):
    cluster2hotspots = generate_cluster_hotspots(self)
    dict_position = {}
    seq_shift = self.sequence[0].find('GGG') - 1 ## for sequence != wtTel22 use
    for resid_i_align in range(1, n_bp+1):  ## ex: 'G4' in wtTel26(='G2' in wtTel22), resid_i=4, resid_i_align=2, resname_i_align='G'
        resid_j_align   = resid_i_align
        resid_i = resid_i_align + seq_shift
        resid_j = resid_j_align + seq_shift
        resname_i_align = self.strandid2sequence['STRAND1'][resid_i]
        resname_j_align = self.strandid2sequence['STRAND1'][resid_j]
        sequence = f'{resname_i_align}{resid_i_align}'
        ## generate dict_hotspot
        cluster = get_cluster(self.mean, resid_i, resid_j)
        hotspots = cluster2hotspots[cluster] 
        if verbose:
            print(self.system, sequence, resid_i, resid_j, cluster, hotspots)
        dict_ks = generate_df_position2hotspots(self, resid_i, resid_j, hotspots)
        dict_position[sequence] = {
            'resid_i': int(resid_i), ## save json format from int istead of np.int64(cannot save) 
            'resid_j': int(resid_j),
            'cluster': cluster,
            'hotspot': dict_ks
        }
    return dict_position

In [6]:
system2offical_name = {
'propeller': 'Propeller',
'basket'   : 'Basket',
'chair'    : 'Chair',
'hybrid-i' : 'Hybrid-I',
'hybrid-ii': 'Hybrid-II',
# 'dsdna'    : 'dsDNA',
}

In [7]:
hs_agent = HotSpotAgent(RB)
hs_agent.load_systems(system2offical_name)

Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/0000_0500/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/0250_0750/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/0500_1000/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/0750_1250/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/1000_1500/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/1250_1750/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fl

/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to 

Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/2000_2500/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/2250_2750/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/2500_3000/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/2750_3250/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/3000_3500/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/3250_3750/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fl

/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to 

Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/4250_4750/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/4500_5000/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/0000_5000/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/basket/d.fluctmatch/bigtraj_fluctmatch/0000_0500/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/basket/d.fluctmatch/bigtraj_fluctmatch/0250_0750/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/basket/d.fluctmatch/bigtraj_fluctmatch/0500_1000/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/basket/d.fluctmatch/bigtraj_fluctmatch/075

...

/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to 

Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii/d.fluctmatch/bigtraj_fluctmatch/0750_1250/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii/d.fluctmatch/bigtraj_fluctmatch/1000_1500/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii/d.fluctmatch/bigtraj_fluctmatch/1250_1750/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii/d.fluctmatch/bigtraj_fluctmatch/1500_2000/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii/d.fluctmatch/bigtraj_fluctmatch/1750_2250/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii/d.fluctmatch/bigtraj_fluctmatch/2000_2500/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii/d.fluctmatch/bigtraj_fl

/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to 

Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii/d.fluctmatch/bigtraj_fluctmatch/3250_3750/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii/d.fluctmatch/bigtraj_fluctmatch/3500_4000/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii/d.fluctmatch/bigtraj_fluctmatch/3750_4250/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii/d.fluctmatch/bigtraj_fluctmatch/4000_4500/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii/d.fluctmatch/bigtraj_fluctmatch/4250_4750/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii/d.fluctmatch/bigtraj_fluctmatch/4500_5000/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii/d.fluctmatch/bigtraj_fl

/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to 

In [8]:
hs_agent.get_hotspot()
hs_agent.get_df_k_with_hotspots()

In [9]:
JSON(hs_agent.dict_hotspot)

<IPython.core.display.JSON object>

In [10]:
hs_agent.df_k_with_hotspots

system,Propeller,Basket,Chair,Hybrid-I,Hybrid-II
sequence,,,,,
A1,13.17 ± 3.86,14.03 ± 0.89,14.21 ± 1.06,12.81 ± 1.87,10.22 ± 1.53
G2,13.40 ± 1.66,11.79 ± 0.38,20.81 ± 0.49,15.72 ± 0.98,18.53 ± 0.66
G3,9.60 ± 3.70,12.66 ± 1.96,12.27 ± 0.93,13.05 ± 1.31,7.78 ± 0.28
G4,13.96 ± 1.04,12.36 ± 2.88,12.66 ± 3.03,14.48 ± 0.49,14.44 ± 1.23
T5,19.40 ± 1.93,19.81 ± 0.31,18.41 ± 2.53,18.29 ± 0.96,15.76 ± 0.61
T6,19.86 ± 1.47,17.53 ± 0.72,17.41 ± 1.32,18.69 ± 0.90,19.23 ± 3.87
A7,13.89 ± 2.78,16.27 ± 4.33,16.10 ± 1.74,15.43 ± 5.85,14.15 ± 2.93
G8,10.48 ± 2.18,16.06 ± 0.65,19.17 ± 0.26,16.52 ± 0.53,19.10 ± 0.60
G9,9.75 ± 2.33,11.72 ± 0.49,18.95 ± 1.03,9.66 ± 1.27,17.87 ± 0.43


In [11]:
system2offical_name = {
'propeller': 'Propeller',
'basket'   : 'Basket',
'chair'    : 'Chair',
'hybrid-i' : 'Hybrid-I',
'hybrid-ii': 'Hybrid-II',
# 'dsdna'    : 'dsDNA',
'basket_na': 'Basket-Na',
'hybrid-ii_wttel26'    : 'Hybrid-II-wtTel26',
'propeller_poh'        : 'Propeller-POH',
'propeller_ewv'        : 'Propeller-EWV', 
'hybrid-ii_wttel26_ewv': 'Hybrid-II-wtTel26-EWV', 
'hybrid-ii_wttel26_poh': 'Hybrid-II-wtTel26-POH', 
}

In [12]:
hs_agent = HotSpotAgent(RB)
hs_agent.load_systems(system2offical_name)
hs_agent.get_hotspot()
hs_agent.get_df_k_with_hotspots()

Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/0000_0500/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/0250_0750/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/0500_1000/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/0750_1250/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/1000_1500/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/1250_1750/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fl

/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to 

Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/1750_2250/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/2000_2500/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/2250_2750/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/2500_3000/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/2750_3250/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/3000_3500/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fl

/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to 

Reading data from: /home/user/workfolder/g4dna_protocol_demo/basket/d.fluctmatch/bigtraj_fluctmatch/1500_2000/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/basket/d.fluctmatch/bigtraj_fluctmatch/1750_2250/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/basket/d.fluctmatch/bigtraj_fluctmatch/2000_2500/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/basket/d.fluctmatch/bigtraj_fluctmatch/2250_2750/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/basket/d.fluctmatch/bigtraj_fluctmatch/2500_3000/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/basket/d.fluctmatch/bigtraj_fluctmatch/2750_3250/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/basket/d.fluctmatch/bigtraj_fluctmatch/3000_3500/re

...

/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to 

Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii_wttel26_poh/d.fluctmatch/bigtraj_fluctmatch/2750_3250/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii_wttel26_poh/d.fluctmatch/bigtraj_fluctmatch/3000_3500/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii_wttel26_poh/d.fluctmatch/bigtraj_fluctmatch/3250_3750/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii_wttel26_poh/d.fluctmatch/bigtraj_fluctmatch/3500_4000/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii_wttel26_poh/d.fluctmatch/bigtraj_fluctmatch/3750_4250/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii_wttel26_poh/d.fluctmatch/bigtraj_fluctmatch/4000_4500/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /ho

/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to 

Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii_wttel26_poh/d.fluctmatch/bigtraj_fluctmatch/0000_5000/result/pairtypes_k_b0_cutoff_5.00.csv


/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))


In [13]:
hs_agent.df_k_with_hotspots

system,Propeller,Basket,Chair,Hybrid-I,Hybrid-II,Basket-Na,Hybrid-II-wtTel26,Propeller-POH,Propeller-EWV,Hybrid-II-wtTel26-EWV,Hybrid-II-wtTel26-POH
sequence,,,,,,,,,,,
A1,13.17 ± 3.86,14.03 ± 0.89,14.21 ± 1.06,12.81 ± 1.87,10.22 ± 1.53,16.24 ± 2.90,17.24 ± 0.43,12.53 ± 3.11,14.90 ± 5.13,12.90 ± 0.85,15.54 ± 1.31
G2,13.40 ± 1.66,11.79 ± 0.38,20.81 ± 0.49,15.72 ± 0.98,18.53 ± 0.66,13.57 ± 1.87,20.62 ± 0.57,11.59 ± 2.35,13.16 ± 1.23,17.35 ± 1.34,14.51 ± 1.76
G3,9.60 ± 3.70,12.66 ± 1.96,12.27 ± 0.93,13.05 ± 1.31,7.78 ± 0.28,12.51 ± 2.96,10.18 ± 0.61,10.58 ± 2.52,11.66 ± 0.42,14.65 ± 0.85,13.14 ± 2.72
G4,13.96 ± 1.04,12.36 ± 2.88,12.66 ± 3.03,14.48 ± 0.49,14.44 ± 1.23,11.02 ± 5.41,13.71 ± 1.82,12.85 ± 0.78,15.32 ± 1.81,10.85 ± 1.47,11.22 ± 2.21
T5,19.40 ± 1.93,19.81 ± 0.31,18.41 ± 2.53,18.29 ± 0.96,15.76 ± 0.61,17.76 ± 3.63,19.46 ± 1.29,20.93 ± 4.12,17.57 ± 1.29,16.00 ± 1.30,16.72 ± 1.75
T6,19.86 ± 1.47,17.53 ± 0.72,17.41 ± 1.32,18.69 ± 0.90,19.23 ± 3.87,14.80 ± 2.50,17.23 ± 0.95,19.16 ± 1.07,19.58 ± 2.26,19.15 ± 0.57,19.13 ± 1.63
A7,13.89 ± 2.78,16.27 ± 4.33,16.10 ± 1.74,15.43 ± 5.85,14.15 ± 2.93,10.93 ± 3.91,17.34 ± 1.93,13.78 ± 2.34,14.80 ± 1.54,12.56 ± 1.24,13.01 ± 1.04
G8,10.48 ± 2.18,16.06 ± 0.65,19.17 ± 0.26,16.52 ± 0.53,19.10 ± 0.60,17.96 ± 1.32,19.43 ± 0.44,12.27 ± 1.22,12.08 ± 2.83,18.51 ± 0.40,18.77 ± 0.54
G9,9.75 ± 2.33,11.72 ± 0.49,18.95 ± 1.03,9.66 ± 1.27,17.87 ± 0.43,12.11 ± 0.84,17.97 ± 0.18,9.55 ± 2.68,8.93 ± 2.09,18.14 ± 0.32,18.30 ± 0.76


### Save all k value data in json format

In [14]:
import json, copy
for subgroup, data in hs_agent.dict_hotspot.items():
    data_copy = copy.deepcopy(data)
    for system, d1 in data_copy.items():
        d1_keys = list(d1.keys())
        for position in d1_keys:
            if isinstance(position, tuple):
                link_position = ','.join(position)
                d1[link_position] = d1.pop(position)
    with open(f'data/{subgroup}_top5.json', 'w') as f:
        json.dump(data_copy, f)

### Save k value with specific position in csv file

In [15]:
df_ks_position = hs_agent.get_df_k_with_hotspots(k_value='k_avg', is_return=True)
df_ks_position.to_csv(f'data/{hs_agent.m_abbr}_avg_top5.csv')

In [16]:
df_ks_position = hs_agent.get_df_k_with_hotspots(k_value='k_std', is_return=True)
df_ks_position.to_csv(f'data/{hs_agent.m_abbr}_std_top5.csv')